# College Admissions and the Stability of Marriage

Author: Jin Yeom (jinyeom@utexas.edu)

In this notebook, we'll solve a little problem my girlfriend's sorority is having: matching new littles and bigs. Fortunately, the same problem has been solved long time ago by mathematicians named David Gale and Lloyd Shapley. They called this problem the **stable marriage problem**, or the **college admissions problem** for more general cases.

In [9]:
import random
from pprint import pprint
from copy import deepcopy

## Stable Marriage Problem
*Given n men and n women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.*

In [14]:
# Example from https://en.wikipedia.org/wiki/Stable_marriage_problem
# The preferences are modified to make the problem a little more challenging.
suiters = ["A", "B", "C"]
reviewers = ["X", "Y", "Z"]
preferences = {
    "A": ["Y", "X", "Z"],
    "B": ["Y", "Z", "X"],
    "C": ["X", "Z", "Y"],
    "X": ["B", "A", "C"],
    "Y": ["C", "B", "A"],
    "Z": ["C", "A", "B"],
}

In [15]:
def gale_shapley(A, B, preferences):
    preferences_ = deepcopy(preferences)
    matches = [False for _ in range(len(B))]
    while False in matches:
        a = A[matches.index(False)]
        b = preferences_[a][0]
        # If b has no match, match it with the current a.
        if b not in matches:
            matches[A.index(a)] = b
        # Otherwise, check if a is more preferable for b than its current match. 
        # Replace with a if so, continue otherwise.
        else:
            a_ = A[matches.index(b)]
            if preferences_[b].index(a) < preferences_[b].index(a_):
                matches[A.index(a_)] = False
                matches[A.index(a)] = b
        del preferences_[a][0]
    return [(a, b) for a, b in zip(A, matches)]

pprint(gale_shapley(suiters, reviewers, preferences))

[('A', 'X'), ('B', 'Y'), ('C', 'Z')]


Now, let's test the algorithm with a larger problem size.

In [11]:
suiters = ["Lynwood", "Rudolph", "Gail", "Dustin", "Claud",
           "Abe", "Mathew", "Delbert", "Gregorio", "Alonzo"]
reviewers = ["Brooke", "Reanna", "Jacquiline", "Marilynn", "Rubi", 
             "Terrie", "Shandra", "Raisa", "Karrie", "Chloe"]
preferences = dict([(s, random.sample(reviewers, len(reviewers))) for s in suiters] +
                   [(r, random.sample(suiters, len(suiters))) for r in reviewers])

pprint(preferences)

{'Abe': ['Brooke',
         'Raisa',
         'Rubi',
         'Chloe',
         'Shandra',
         'Terrie',
         'Marilynn',
         'Reanna',
         'Karrie',
         'Jacquiline'],
 'Alonzo': ['Raisa',
            'Reanna',
            'Karrie',
            'Chloe',
            'Jacquiline',
            'Brooke',
            'Terrie',
            'Marilynn',
            'Shandra',
            'Rubi'],
 'Brooke': ['Rudolph',
            'Delbert',
            'Claud',
            'Mathew',
            'Gail',
            'Abe',
            'Lynwood',
            'Gregorio',
            'Dustin',
            'Alonzo'],
 'Chloe': ['Dustin',
           'Lynwood',
           'Gail',
           'Abe',
           'Gregorio',
           'Claud',
           'Mathew',
           'Alonzo',
           'Rudolph',
           'Delbert'],
 'Claud': ['Rubi',
           'Shandra',
           'Jacquiline',
           'Marilynn',
           'Raisa',
           'Terrie',
           'Chloe',
  

In [12]:
pprint(gale_shapley(suiters, reviewers, preferences))

[('Lynwood', 'Karrie'),
 ('Rudolph', 'Chloe'),
 ('Gail', 'Jacquiline'),
 ('Dustin', 'Terrie'),
 ('Claud', 'Marilynn'),
 ('Abe', 'Rubi'),
 ('Mathew', 'Brooke'),
 ('Delbert', 'Shandra'),
 ('Gregorio', 'Reanna'),
 ('Alonzo', 'Raisa')]


## College Admissions Problem